# 00 - Control of the Quality of the Data

**Objectives**: Validate the completeness of the database available from https://github.com/JeffSackmann/tennis_atp, before actually starting the work.

In order to model player evolution using the Zermelo algorithms, it must be ensured that the `.csv` files contain consistent data across all years: we want to have _ATP_, _Qualifying & Challenger_ and _Futures_ data. It must also be verified that column names remain identic. Checking for missing values in columns is also done here.

In [1]:
import os

import pandas as pd
pd.set_option('display.max_colwidth', None) # to display full content of dataframe

from tqdm import tqdm

In [2]:
data_path="../data/tennis_atp"

if os.path.exists(data_path):
    print(f"The folder exists!")
else: 
    print("The folder can't be found. Verify the path.")

The folder exists!


## Check for Temporal Completeness

We scan a large period of years to identify which ones have these three files availables:
1. **Main Tour**: `atp_matches_YEAR.csv` (tour-level main draw matches) 

2. **Intermediate**: `atp_matches_qual_chall_YEAR.csv` (tour-level qualifying and challenger main draw matches)

3. **Futures**: `atp_matches_futures_YEAR.csv` (promising youngsters)


This will define our valide timeframe for the future analysis.

In [3]:
years = range(1950, 2026)
all_files = os.listdir(data_path)

complete_years=[]

for year in years:
    files_year = [file for file in all_files if str(year) in file and file.endswith(".csv")]

    file_main = f"atp_matches_{year}.csv" in files_year
    file_qual_chall = f"atp_matches_qual_chall_{year}.csv" in files_year
    file_future = f"atp_matches_futures_{year}.csv" in files_year

    if file_main and file_qual_chall and file_future:
        complete_years.append(year)

# print(complete_years)
# if complete_years == list(range(1991,2025)):
#     print("All years betwen 1991 and 2024 are complete!")

print("Result: All files are available for this period -> 1991-2024")

Result: All files are available for this period -> 1991-2024


In [4]:
start_year = 1991
end_year = 2024

## Stability of the Columns over Years and Checking Missing Values

The objective is to determine which variables are consistently available throughout the entire period across all type of games (ATP, Challengers, Futures). This is done by counting the number of appearances of each column name across all years and files.

We also verify the contents of the columns, to ensure they are not empty. We want to be sure that the player IDs are present 100% of the time, for example.


In [5]:
# creation of a dictionary, counting the number of appearances of each column name
columns_count = {}

# dictionary for knowing the number of Nan in each column
nan_count = {}

files_count = 0
rows_count = 0

# list all files in the data folder
files = os.listdir(data_path)

for year in range(start_year, end_year+1):
    
    # get the files we consider for a specific year
    target_files = [file for file in files if str(year) in file and file.endswith(".csv") and "double" not in file]

    # loop over each selected file
    for file in target_files:
        file_path = os.path.join(data_path, file)

        # reading the file
        df = pd.read_csv(file_path, low_memory=False)

        # updating counts
        files_count +=1
        rows_count += len(df)

        # counting the appearances of each column
        for column in df.columns:
            if column in columns_count:
                columns_count[column] += 1
            else:
                columns_count[column] = 1

        # counting the number of NaN in each column
        nans = df.isnull().sum()
        
        for column, count in nans.items():
            if column in nan_count:
                nan_count[column] += count
            else:
                nan_count[column] = count


# check which columns are complete (present in all files)
complete_columns = []
uncomplete_columns = []

for column, count in columns_count.items():
    if count == files_count:
        complete_columns.append(column)
    else:
        uncomplete_columns.append(f"{column} ({count/files_count*100}%)")

print(f"Number of files analysed: {files_count}")
print(f"Complete columns (available in all files): {len(complete_columns)}/{len(columns_count)}")

if uncomplete_columns:
    print(f"Uncomplete columns: {uncomplete_columns}")

columns_table = pd.DataFrame(list(columns_count.items()), columns=["Column", "Count"])
columns_table["%"] = (columns_table["Count"] / files_count * 100)
columns_table = columns_table.sort_values(by="%", ascending=False, ignore_index=True)
# display(columns_table)

missing_data = []

for column in complete_columns:
    missing_values_count = nan_count.get(column, 0)
    pourcentage_missing = (missing_values_count / rows_count) * 100

    missing_data.append({"Column": column, "NaN Count": missing_values_count, "% Data Missing": f"{pourcentage_missing:.2f}"})

df_missing = pd.DataFrame(missing_data)
df_missing = df_missing.sort_values(by="NaN Count", ascending=False, ignore_index=True)
display(df_missing)

Number of files analysed: 102
Complete columns (available in all files): 49/49


,Column,NaN Count,% Data Missing
0,winner_entry,659257,81.53
1,loser_seed,621190,76.82
2,minutes,601485,74.38
3,l_SvGms,594460,73.51
4,w_SvGms,594460,73.51
5,w_df,594458,73.51
6,l_svpt,594458,73.51
7,w_bpFaced,594458,73.51
8,l_1stIn,594458,73.51
9,w_ace,594458,73.51


## Content Inspection

To build a realistic model later, we need to understand the content of our dataset. We inspect here the columns content for the validated columns.

In [6]:
def get_column_content(data: pd.DataFrame,
                       limit: int=10,
                       excluded_columns: list=None
                       ) -> pd.DataFrame:

    """
    Get a summary of the content of specified columns in a DataFrame.

    Args:
        data (pd.DataFrame): The input DataFrame.
        limit (int): If number of unique values in a column is less than this threshold, display all unique values. Else, display a summary.
        excluded_columns (list): List of columns to exclude from the complete analysis. (If a column is excluded, it will only show the summary with top 3 values.)

    Returns:
        pd.DataFrame: A DataFrame containing the summary of the specified columns.
    """
    if excluded_columns is None:
        excluded_columns = []

    # initialise an empty list to store summary data
    summary_data=[]

    for column in data.columns: 
        content_unique = data[column].nunique() # number of unique values 
        percentage_missing = data[column].isnull().mean() * 100 # percentage of missing values
        ratios = data[column].value_counts(normalize=True) # counts of each unique value (stored in a Series of type value: ratio)


        if content_unique <= limit and column not in excluded_columns: # show all unique values
            elements = [f"{val} ({round(ratio * 100, 2)}%)" for val, ratio in ratios.items()]
            text = " | ".join(elements)

        else: # show summary with only top 3 values appearing
            top_3_elements = [f"{val} ({round(ratio * 100, 2)}%)" for val, ratio in ratios.head(3).items()]
            text = " | ".join(top_3_elements)

            # add indication of other values
            other_counts = content_unique - 3
            text += f" + {other_counts} others"
            
        # append the summary for the current column to the list
        summary_data.append({ "Column": column, 
                         "Number of different values": content_unique,
                         "% Data Missing": round(percentage_missing, 2),
                         "Content (Note: % doesn't include Missing Data)": text})


    df_summary = pd.DataFrame(summary_data)
    df_summary = df_summary.sort_values(by="Number of different values", ascending=True, ignore_index=True)

    return df_summary

A dataframe is created, containing only the validated columns, for all years at once. With this, we can then apply the `get_column_content` function to inspect the content of each column.

In [7]:
all_data = []

for year in tqdm(range(start_year, end_year+1), desc="Loading data by year"):

    # get the files we consider for a specific year
    target_files = [file for file in files if str(year) in file and file.endswith(".csv") and "double" not in file]

    # loop over each selected file
    for file in target_files:
        file_path = os.path.join(data_path, file)

        df_all = pd.read_csv(file_path, low_memory=False)

        df_complete_columns = df_all[complete_columns]

        all_data.append(df_complete_columns)

# concatenate all dataframes in the list into a single dataframe
all_data = pd.concat(all_data, ignore_index=True)

if not all_data.empty:
    print(f"Full dataset created! Shape -> {all_data.shape}")

else:
    print("Warning: No file was loaded!")

Loading data by year: 100%|██████████| 34/34 [00:01<00:00, 17.96it/s]


Full dataset created! Shape -> (808627, 49)


In [8]:
get_column_content(all_data, limit=26, excluded_columns=["draw_size", "w_bpSaved", "l_bpSaved", "w_df"])

,Column,Number of different values,% Data Missing,Content (Note: % doesn't include Missing Data)
0,best_of,2,0.00,3 (97.04%) | 5 (2.96%)
1,winner_hand,4,0.00,R (80.35%) | L (11.58%) | U (8.03%) | A (0.04%)
2,loser_hand,4,0.00,R (74.6%) | U (14.82%) | L (10.53%) | A (0.04%)
3,surface,4,0.01,Clay (48.16%) | Hard (44.92%) | Carpet (4.21%) | Grass (2.72%)
4,tourney_level,10,0.00,S (53.15%) | C (21.18%) | A (9.61%) | 15 (5.3%) | G (3.48%) | 25 (3.2%) | M (2.78%) | D (1.22%) | F (0.07%) | O (0.01%)
5,round,14,0.00,R32 (44.61%) | R16 (22.87%) | QF (11.43%) | SF (5.72%) | Q1 (3.82%) | R64 (3.16%) | F (2.86%) | Q2 (2.12%) | RR (1.4%) | R128 (1.3%) | Q3 (0.58%) | Q4 (0.12%) | ER (0.0%) | BR (0.0%)
6,draw_size,24,0.00,32 (84.2%) | 128 (3.81%) | 64 (3.63%) + 21 others
7,w_bpSaved,25,73.51,0.0 (16.47%) | 2.0 (14.88%) | 1.0 (14.37%) + 22 others
8,winner_entry,25,81.53,Q (60.31%) | WC (27.23%) | LL (4.98%) | SE (3.23%) | PR (1.81%) | Alt (1.42%) | JE (0.28%) | ITF (0.21%) | ALT (0.16%) | JR (0.12%) | IR (0.11%) | A (0.06%) | S (0.02%) | AL (0.02%) | JR (0.02%) | P (0.02%) | 3) (0.0%) | 6) A (0.0%) | 6) (0.0%) | 5) A (0.0%) | 1) (0.0%) | 2) A (0.0%) | I (0.0%) | UP (0.0%) | W (0.0%)
9,w_df,25,73.51,1.0 (21.66%) | 2.0 (20.37%) | 3.0 (15.51%) + 22 others


There are different types of data:

### Tournament information:
- `best of` = maximum number of sets played in the match: **3** (2 sets needed to win) or **5** (3 sets needed to win).

- `surface` = type of court: **Hard**, **Clay**, **Grass**, **Carpet**.

- `tourney_level` = level of the tournament: 

    - The elite: **G** = Grand Slams (Major tournaments, e.g. Australian Open, Roland Garros, Wimbledon, US Open)

    - The main circuit (ATP Tour): **M** = Masters 1000s, **F** = Season-ending championships, includes both the ATP Finals and NextGen Finals (U21), **A** = ATP others (ATP500 and ATP250)

    - International and team events: **D** = Davis Cup, **O** = Olympics 

    - Intermediate level: **C** = Challengers

    - Entry level: **S** = Satellites/Futures, **15** = M15 (ITF $15,000 tournaments), **25** = M25 (ITF $25,000 tournaments)

- `draw_size`: Number of players in the draw (usually 32, 64, or 128)

- `tourney_id` = identifier for the tournament, regrouping matches of the same event (e.g. "2017-560")
- `tourney_name` = readable name of the tournament (e.g. "Australian Open", "M15 Monastir")
- `tourney_date` = eight digits usually corresponding to the Monday of the tournamenet week (format YYYYMMDD)

### Current Match information:

- `round` = round of the tournament: 

    - Main draw: **F** = Final, **SF** = Semi-Final, **QF** = Quarter-Final, **R16** = Round of 16, **R32** = Round of 32, **R64** = Round of 64, **R128** = Round of 128

    - Qualifiers: **Q1**, **Q2**, **Q3**, **Q4** = Qualifying rounds 1, 2, 3 and 4

    - Special cases: **RR** = Round Robin (used e.g. in ATP Finals, when every player plays every other player), **BR** = Bronze Medal Match (used in Olympics for the 3rd place)

- `score` = final score, looking like "6-4 6-4"
- `minutes` = duration of the match (missing for about 75% of the matches!)
- `match_num` = specific match identifier (often starting from 1, sometimes counting down from 300 and sometimes arbitrary)

### Player profiles ("winner_" for winner, "loser_" for loser):

- `_hand` = playing hand of the player: **R** = right-handed, **L** = left-handed, **U** = unknown, **A** = ambidextrous

- `_id` = unique ID of the player in the database (e.g. "103819", useful for tracking players, because name can have typos)
- `_name`= player's full name
- `_ht` = height in cm (missing for about 40% of players, mostly lower-ranked)
- `_age` = age in years, as of the  _tourney\_date_ (e.g. 21.3)
- `_ioc` = 3-caracter country code (e.g. ESP, USA, ...)
- `_rank` = ATP rank (as of the tourney_date or the most recent ranking date before the _tourney\_date_)
- `_rank_points` = number of ATP ranking points
- `_seed` = seed number in the tournament (high missing rate normal, because only top players are seeded)
- `_entry_`= how the player entered the tournament (high missing rate normal, because of direct acceptance --> a player can enter thanks to his ranking):
    - **WC** = wild card: the player didn't have the ranking to enter directly, but the tournament invited him anyway (local player, famous player returning from injury)

    - **Q** = qualifier: the player didn't have a high enough ranking to enter directly --> he had to play and win the qualification tournament before the main event
    - **LL** = player who lost in the final round of the qualification but was drafted into the main draw to replace a player who withdrew (injury/illness) at the last moment
    - **PR** = protected ranking: A player returning from a long-term injury (6+ months), he is allowed to use his old ranking to enter tournaments for a limited time
    - **SE** = special exempt: the player was unable to play the qualifying tournament because he was still competing in the semi-finals or finals of another tournament the previous week
    - **ALT** = alternate:  A player who replaces a withdrawal based on the ranking list (similar to LL, but without necessarily playing qualifiers first)

   -  **JE** = junior exempt: top-ranked ITF juniors player to help them transition to professional tournaments
   - **ITF / IR** = ITF Entry: player entered based on his ITF ranking
   - **JR** = juinior ranked player: generic code for a junior player entry (?)
   - Other strange codes (errors?): _A, AL, P, S, 1), 2) A, 3), 5) A, 6) A, 6), 8), I, J, UP, W_



  








### Matchs statistics ("w_" for winner, "l_" for loser):

- `_ace`= number of aces

- `_df` = number of double faults
- `_svpt` = total service points played (total number of points where the player was serving)
- `_1stIn` = number of first serves in
- `_1stWon` = number of points won on first serve
- `_2ndWon` = number of points won on second serve
- `_SvGms` = number of serve games played
- `_bpFaced` = total number of break points (= times the opponent could have won a game on the player's serve) faced during the match
- `_bpSaved` = total number of break points saved during the match

_Warning: 75% of the matches have missing values for these statistics, mostly the Futures tournaments._

**Important note**: During exploration of the dataset, I found some errors:

- typo errors (e.g., "Us Open" vs "US Open" for tournament names)

- _\_entry_ column has strange codes that do not correspond to known entry types (e.g. 6) A, 3), I, J, UP)
- ...

For the moment, these inconsistencies are tolerated, as they should not impact the computation of Zermelo strengths (relying on player IDs, i.e. _winner\_id_, _loser\_id_). Therefore, they will be addressed later if needed for specific columns (surface or tournament name for example).
    

## Creation of a unique DataFrame

The idea is to create a single DataFrame containing all validated columns, for all years at once. Two columns are added to keep track of the year and the type of tournament (ATP, Qualifiers/Challengers, Futures).

In [9]:
all_matches_1991_2024_list = []

for year in tqdm(range(start_year, end_year+1), desc="Loading data by year"):

     # get the files we consider for a specific year
    target_files = [file for file in files if str(year) in file and file.endswith(".csv") and "double" not in file]

    # loop over each selected file
    for file in target_files:
        file_path = os.path.join(data_path, file)

        df_file = pd.read_csv(file_path, low_memory=False)

        # adding a column to know the year of the games
        df_file["year"] = year

        # adding a column to know the type of tournament: ATP Main, Challengers/Qual, Futures
        if "qual_chall" in file:
            df_file["type"] = "QC"
        elif "futures" in file:
             df_file["type"] = "F"
        else:
            df_file["type"] = "ATP"
                    
        all_matches_1991_2024_list.append(df_file)

# creating the full dataset
all_matches_1991_2024_data = pd.concat(all_matches_1991_2024_list, ignore_index=True)

print("Full dataset has been loaded!")
print("Total number of matches available (rows): ", all_matches_1991_2024_data.shape[0])

# look at the number of matches in each "type" (ATP, QC, F)
type_table = all_matches_1991_2024_data["type"].value_counts().reset_index()
type_table.columns = ["Type", "Number of matches"]
display(type_table)

Loading data by year: 100%|██████████| 34/34 [00:01<00:00, 20.58it/s]


Full dataset has been loaded!
Total number of matches available (rows):  808627


,Type,Number of matches
0,F,498555
1,QC,201697
2,ATP,108375


In [11]:
# saving the full dataset in a .csv file
output_folder = "../data/processed"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"New folder created! (location: {output_folder})")

output_file = os.path.join(output_folder, "all_matches_1991-2024.csv")

if os.path.exists(output_file):
    print(f"File ({output_file}) already exists! It will be overwritten...")

else:
    print(f"Creating file ({output_file})...")

all_matches_1991_2024_data.to_csv(output_file, index=False)
print("File saved!")

File (../data/processed/all_matches_1991-2024.csv) already exists! It will be overwritten...
File saved!
